# これは何？  
pythonで機械学習するためのチュートリアルです.csvファイルの読み込み方，機械学習ライブラリの使い方，図のプロットのしかたがわかるように作りました．  
上から順に実行していけば良いはずです．途中で動かなかったらライブラリが入ってない可能性があります．  
わからないところがあったらまずはそこの関数名でググってみてください．

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import re
%matplotlib inline

# データ整形

In [2]:
data = pd.read_csv("tutorial.csv",encoding="shift-jis",na_values="")
print("データの形")
print(data.shape)
data.head()

データの形
(29918, 4)


,order_of_finish,popularity,horse_weight,tanshou_odds
0,2,4,412,15.8
1,6,2,418,6.9
2,8,9,458,145.4
3,7,5,408,19.8
4,4,8,442,102.3


In [3]:
#Nullチェック
data.isnull().sum()

order_of_finish    0
popularity         0
horse_weight       0
tanshou_odds       0
dtype: int64

各列の値をチェックしてみる

In [4]:
data["order_of_finish"].value_counts()

1     2081
2     2038
3     2036
4     2036
5     2030
6     2030
7     2029
8     2024
9     1994
10    1922
11    1824
12    1722
13    1573
14    1404
15    1213
16     936
       324
0      306
17     221
18     175
Name: order_of_finish, dtype: int64

In [5]:
data["popularity"].value_counts()

5     2049
3     2049
2     2048
4     2046
6     2045
7     2041
1     2040
8     2034
9     1997
10    1949
11    1843
12    1739
13    1598
14    1428
15    1235
16     974
17     229
18     193
0      111
80      14
20      13
70      13
40      12
71      11
50      10
21       9
31       9
19       8
91       8
51       8
      ... 
97       2
86       2
63       2
98       2
32       2
47       2
39       2
65       2
87       2
64       2
58       2
62       2
25       2
88       2
36       2
74       1
59       1
46       1
72       1
94       1
52       1
45       1
28       1
26       1
76       1
48       1
         1
53       1
89       1
95       1
Name: popularity, Length: 89, dtype: int64

In [6]:
data["horse_weight"].value_counts()

470    865
480    806
472    798
478    773
464    767
468    756
476    756
474    747
466    742
460    722
462    718
482    705
454    698
484    693
456    691
486    686
458    681
450    661
452    654
488    652
490    641
448    601
444    592
492    586
446    551
496    541
494    533
500    524
498    510
442    501
      ... 
751      1
362      1
665      1
52       1
949      1
148      1
212      1
324      1
596      1
612      1
644      1
900      1
165      1
245      1
453      1
645      1
150      1
153      1
310      1
582      1
598      1
854      1
950      1
551      1
647      1
951      1
24       1
264      1
360      1
375      1
Name: horse_weight, Length: 231, dtype: int64

In [7]:
data["horse_weight"].value_counts()

470    865
480    806
472    798
478    773
464    767
468    756
476    756
474    747
466    742
460    722
462    718
482    705
454    698
484    693
456    691
486    686
458    681
450    661
452    654
488    652
490    641
448    601
444    592
492    586
446    551
496    541
494    533
500    524
498    510
442    501
      ... 
751      1
362      1
665      1
52       1
949      1
148      1
212      1
324      1
596      1
612      1
644      1
900      1
165      1
245      1
453      1
645      1
150      1
153      1
310      1
582      1
598      1
854      1
950      1
551      1
647      1
951      1
24       1
264      1
360      1
375      1
Name: horse_weight, Length: 231, dtype: int64

いくつかの問題点  
1.想定してないものが入っている時がある（元のDBを見てみるとtanshou_oddsに文字が入ってたりしている)　　
2.人気順がおかしい(0とか80とか)行がある  
3.馬体重がおかしい（50kgとか900kgとか)行がある  
対処  
1.数字以外を含む行は消す
2.1~18以外の数字の行を削除(適当)  
3.今回は触らないことにする

In [8]:
data = data[data["order_of_finish"].str.contains("(\d)+")]
data = data[data["popularity"].str.contains("(\d)+")]
data = data[data["tanshou_odds"].str.contains("(\d)+")]
print(data.shape)

(29488, 4)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
#文字列を数値に変換した行を作る
data.loc[:,"order_of_finish_num"] = data.loc[:,"order_of_finish"].astype(int)
data.loc[:,"popularity_num"] = data.loc[:,"popularity"].astype(int)
data.loc[:,"horse_weight_num"] = data.loc[:,"horse_weight"].astype(int)
data.loc[:,"tanshou_odds_num"] = data.loc[:,"tanshou_odds"].astype(float)

print(data.shape)
data.head()

(29488, 8)


,order_of_finish,popularity,horse_weight,tanshou_odds,order_of_finish_num,popularity_num,horse_weight_num,tanshou_odds_num
0,2,4,412,15.8,2,4,412,15.8
1,6,2,418,6.9,6,2,418,6.9
2,8,9,458,145.4,8,9,458,145.4
3,7,5,408,19.8,7,5,408,19.8
4,4,8,442,102.3,4,8,442,102.3


In [10]:
#2の処理
data =  data.loc[(data.loc[:,"popularity_num"]>=1) & (data.loc[:,"popularity_num"]<=18),:]
print(data.shape)

(29389, 8)


In [11]:
data.head()

,order_of_finish,popularity,horse_weight,tanshou_odds,order_of_finish_num,popularity_num,horse_weight_num,tanshou_odds_num
0,2,4,412,15.8,2,4,412,15.8
1,6,2,418,6.9,6,2,418,6.9
2,8,9,458,145.4,8,9,458,145.4
3,7,5,408,19.8,7,5,408,19.8
4,4,8,442,102.3,4,8,442,102.3


単勝した馬とそうでない馬を示すtanshou_flagをつくる

In [12]:
data.loc[:,"tanshou_flag"] = (data.loc[:,"order_of_finish_num"] == 1)
data.head(20)

,order_of_finish,popularity,horse_weight,tanshou_odds,order_of_finish_num,popularity_num,horse_weight_num,tanshou_odds_num,tanshou_flag
0,2,4,412,15.8,2,4,412,15.8,False
1,6,2,418,6.9,6,2,418,6.9,False
2,8,9,458,145.4,8,9,458,145.4,False
3,7,5,408,19.8,7,5,408,19.8,False
4,4,8,442,102.3,4,8,442,102.3,False
5,3,3,460,14.5,3,3,460,14.5,False
6,5,7,452,97.2,5,7,452,97.2,False
7,0,6,470,49.9,0,6,470,49.9,False
8,1,1,492,1.1,1,1,492,1.1,True
9,8,9,484,61.9,8,9,484,61.9,False


# 学習

popularity,horse_weightからtanshou_flagを予測してみる  
まず，テストデータとトレインデータに分割する(2:8)

In [13]:
features = ["popularity_num","horse_weight"]
target = ["tanshou_flag"]

X = data
y = data.loc[:,target]
print("デザインマトリックスの形")
print(X.shape)
print("ラベルマトリックス")
print(y.shape)

デザインマトリックスの形
(29389, 9)
ラベルマトリックス
(29389, 1)


In [14]:
data_train, data_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print("トレインデータ")
print(data_train.shape)
print(y_train.shape)
print("テストデータ")
print(data_test.shape)
print(y_test.shape)

トレインデータ
(23511, 9)
(23511, 1)
テストデータ
(5878, 9)
(5878, 1)


In [15]:
#学習に使う列だけ選択
X_train = data_train.loc[:,features]
X_test = data_test.loc[:,features]

In [16]:
#DataFrameのまま扱うとindexが保持されて便利
X_train.head()

,popularity_num,horse_weight
9467,13,464
11373,13,472
2285,8,492
7872,5,418
8997,8,462


In [17]:
#tutrial http://scikit-learn.org/stable/modules/svm.html
# SVC()の説明　http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC
clf = svm.SVC()
clf.fit(X_train, y_train)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## 評価

In [18]:
train_score = clf.score(X_train,y_train)
test_score = clf.score(X_test,y_test)
print("train dataのscore:", train_score)
print("test dataのscore:", test_score)

train dataのscore: 0.932074348177
test dataのscore: 0.927696495407



コンフュージョンマトリックスを描いてみる

In [19]:
train_y_pred = clf.predict(X_train)
test_y_pred = clf.predict(X_test)

In [20]:
print("train data の　予測の内訳")
print(pd.Series(train_y_pred).value_counts())
print("test data の　予測の内訳")
print(pd.Series(test_y_pred).value_counts())

train data の　予測の内訳
False    23454
True        57
dtype: int64
test data の　予測の内訳
False    5865
True       13
dtype: int64


In [21]:
#行が真のクラス(F,T),列が予想クラス(N,P)
print("confusion_matrix in train")
print(confusion_matrix(y_train,train_y_pred))
print("confusion_matrix in test")
print(confusion_matrix(y_test,test_y_pred))

confusion_matrix in train
[[21874    17]
 [ 1580    40]]
confusion_matrix in test
[[5447    7]
 [ 418    6]]


[FN FP  
 TN TP]  


以下回収率の計算

In [22]:

data_train.loc[:,"predict"] = pd.Series(train_y_pred,index=data_train.index)
print(data_train.shape)
data_train.head()

(23511, 10)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,order_of_finish,popularity,horse_weight,tanshou_odds,order_of_finish_num,popularity_num,horse_weight_num,tanshou_odds_num,tanshou_flag,predict
9467,16,13,464,47,16,13,464,47.0,False,False
11373,7,13,472,69.6,7,13,472,69.6,False,False
2285,11,8,492,40.8,11,8,492,40.8,False,False
7872,5,5,418,12.6,5,5,418,12.6,False,False
8997,3,8,462,69.4,3,8,462,69.4,False,False


In [23]:
data_test.loc[:,"predict"] = pd.Series(test_y_pred,index=data_test.index)
print(data_test.shape)
data_test.head()

(5878, 10)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,order_of_finish,popularity,horse_weight,tanshou_odds,order_of_finish_num,popularity_num,horse_weight_num,tanshou_odds_num,tanshou_flag,predict
14329,10,9,466,28.4,10,9,466,28.4,False,False
13460,1,1,536,3.8,1,1,536,3.8,True,False
23765,4,4,510,12.6,4,4,510,12.6,False,False
5258,5,9,516,23.9,5,9,516,23.9,False,False
26040,14,8,434,23.1,14,8,434,23.1,False,False


In [24]:
#単勝した馬のみのデータ
data_train_T = data_train.loc[data_train["tanshou_flag"] == True,:]
print(data_train_T.shape)
data_train_T.head()

(1620, 10)


,order_of_finish,popularity,horse_weight,tanshou_odds,order_of_finish_num,popularity_num,horse_weight_num,tanshou_odds_num,tanshou_flag,predict
27573,1,2,460,2.9,1,2,460,2.9,True,False
21919,1,1,484,2.5,1,1,484,2.5,True,False
26342,1,2,508,2.8,1,2,508,2.8,True,False
29867,1,9,496,23,1,9,496,23.0,True,False
23840,1,1,484,2.6,1,1,484,2.6,True,False


In [25]:
#そのなかで単勝すると予測したデータ
data_train_TP = data_train_T.loc[data_train_T["predict"] == True,:]
print(data_train_TP.shape)
data_train_TP.head()

(40, 10)


,order_of_finish,popularity,horse_weight,tanshou_odds,order_of_finish_num,popularity_num,horse_weight_num,tanshou_odds_num,tanshou_flag,predict
9619,1,1,564,2,1,1,564,2.0,True,True
10876,1,1,538,1.8,1,1,538,1.8,True,True
16296,1,10,651,33.,1,10,651,33.0,True,True
26790,1,1,520,2.7,1,1,520,2.7,True,True
18696,1,2,416,3,1,2,416,3.0,True,True


In [26]:
#TPの単勝オッズの合計/単勝と予測したデータ数
train_kaisyuuritu = data_train_TP["tanshou_odds_num"].sum()/(data_train.loc[data_train["predict"]==True,:].shape[0])
print("train dataの回収率は",train_kaisyuuritu)

train dataの回収率は 10.222807017543861


In [27]:
data_train_TP["tanshou_odds_num"]

9619       2.0
10876      1.8
16296     33.0
26790      2.7
18696      3.0
15459      2.1
23985      2.5
11728      3.4
11215      3.6
13298    342.0
20781      2.2
6029       2.3
21376     99.0
5378       1.7
13587      2.6
8015       1.8
8288       5.6
10300      1.3
5981       2.4
21461      3.0
16800      3.7
16248      1.7
8068       4.9
6993       2.0
3187       2.5
3889       1.4
7605       2.0
7547       1.7
3468       4.3
20684      4.6
1087       4.6
12420     10.2
27650      2.8
9739       3.7
20712      2.3
18351      1.8
9693       3.6
9198       2.1
20802      3.4
21404      1.4
Name: tanshou_odds_num, dtype: float64

In [30]:
data_test_T = data_test.loc[data_test["tanshou_flag"] == True,:]
print(data_test_T.shape)
data_test_TP = data_test_T.loc[data_test_T["predict"] == True,:]
print(data_test_TP.shape)
test_kaisyuuritu = data_test_TP["tanshou_odds_num"].sum()/(data_test.loc[data_test["predict"]==True,:].shape[0])
print("test dataの回収率は",test_kaisyuuritu)

(424, 10)
(6, 10)
test dataの回収率は 1.3230769230769233


In [31]:
data_test_TP

,order_of_finish,popularity,horse_weight,tanshou_odds,order_of_finish_num,popularity_num,horse_weight_num,tanshou_odds_num,tanshou_flag,predict
26822,1,1,538,2.7,1,1,538,2.7,True,True
13781,1,2,558,3.2,1,2,558,3.2,True,True
24895,1,1,538,2.5,1,1,538,2.5,True,True
6166,1,1,520,3,1,1,520,3.0,True,True
19752,1,1,528,3.7,1,1,528,3.7,True,True
19951,1,1,520,2.1,1,1,520,2.1,True,True


あれ？回収率1超えた？(要調査)